In [ ]:
# 🚀 MULTI-MODEL SERVING ENDPOINT (GIT VARIABLE DRIVEN) - FIXED

import os
import json
import time
import yaml
import traceback
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import (
    EndpointCoreConfigInput,
    ServedEntityInput,
    TrafficConfig,  
    Route 
)
import mlflow
from mlflow.tracking import MlflowClient

print("=" * 80)
print("🚀 MULTI-MODEL SERVING (GIT VARIABLE CONTROLLED)")
print("=" * 80)

# --------------------------------------------------
# 1️⃣ LOAD PIPELINE CONFIG
# --------------------------------------------------
try:
    with open("pipeline_config.yml", "r") as f:
        pipeline_cfg = yaml.safe_load(f)
    print("✅ pipeline_config.yml loaded")
except Exception as e:
    print("❌ Failed to load pipeline_config.yml")
    traceback.print_exc()
    raise e

try:
    with open("experiments_config.yml", "r") as f:
        experiments_cfg = yaml.safe_load(f)
    print("✅ experiments_config.yml loaded")
except Exception as e:
    print("❌ Failed to load experiments_config.yml")
    traceback.print_exc()
    raise e

# --------------------------------------------------
# 2️⃣ CREATE WIDGET FOR PARAMETER
# --------------------------------------------------
try:
    dbutils.widgets.text("MODEL_SERVING_CONFIG", "{}", "Model Serving Config (JSON)")
except:
    pass

# --------------------------------------------------
# 3️⃣ LOAD MODEL SERVING CONFIG FROM GIT VARIABLE
# --------------------------------------------------
MODEL_SERVING_CONFIG_RAW = None

try:
    MODEL_SERVING_CONFIG_RAW = dbutils.widgets.get("MODEL_SERVING_CONFIG")
    print(f"📌 MODEL_SERVING_CONFIG from Widget")
except:
    MODEL_SERVING_CONFIG_RAW = os.getenv("MODEL_SERVING_CONFIG", "{}")
    print(f"📌 MODEL_SERVING_CONFIG from ENV")

if not MODEL_SERVING_CONFIG_RAW or MODEL_SERVING_CONFIG_RAW.strip() == "{}":
    raise ValueError(
        "❌ MODEL_SERVING_CONFIG Git variable not set!\n"
        "\n"
        "Expected format (single line, no newlines):\n"
        '{"random_forest":{"model_name":"workspace.ml_credit_risk.credit_risk_model_random_forest","version":"3","traffic":70,"active":true},"xgboost":{"model_name":"workspace.ml_credit_risk.credit_risk_model_xgboost","version":"2","traffic":30,"active":true}}\n'
        "\n"
        "OR use shorthand without full model name (auto-generated):\n"
        '{"random_forest":{"version":"3","traffic":70,"active":true},"xgboost":{"version":"2","traffic":30,"active":true}}'
    )

# ✅ Parse JSON
try:
    MODEL_SERVING_CONFIG = json.loads(MODEL_SERVING_CONFIG_RAW)
    print("\n✅ MODEL_SERVING_CONFIG parsed successfully:")
    print(json.dumps(MODEL_SERVING_CONFIG, indent=2))
except json.JSONDecodeError as e:
    raise ValueError(
        f"❌ Invalid JSON in MODEL_SERVING_CONFIG:\n{e}\n"
        f"\nReceived:\n{MODEL_SERVING_CONFIG_RAW}\n"
        f"\nMake sure it's valid JSON (use online JSON validator)"
    )

# --------------------------------------------------
# 4️⃣ VALIDATE AND AUTO-COMPLETE MODEL NAMES
# --------------------------------------------------
def get_full_model_name(model_type: str) -> str:
    """
    Generate full Unity Catalog model name from model_type
    """
    UC_CATALOG = pipeline_cfg["models"]["catalog"]
    UC_SCHEMA = pipeline_cfg["models"]["schema"]
    BASE_NAME = pipeline_cfg["models"]["base_name"]
    NAMING_FMT = pipeline_cfg["models"]["naming"]["format"]
    
    return NAMING_FMT.format(
        catalog=UC_CATALOG,
        schema=UC_SCHEMA,
        base_name=BASE_NAME,
        model_type=model_type
    )

# ✅ Validate model types and auto-complete model_name if missing
available_models = list(experiments_cfg.get("models", {}).keys())

for model_type, cfg in MODEL_SERVING_CONFIG.items():
    
    # Validate model_type exists in experiments_config.yml
    if model_type not in available_models:
        raise ValueError(
            f"❌ Invalid model_type '{model_type}' in MODEL_SERVING_CONFIG\n"
            f"   Available models: {available_models}"
        )
    
    # ✅ Auto-complete model_name if not provided
    if "model_name" not in cfg or not cfg["model_name"]:
        cfg["model_name"] = get_full_model_name(model_type)
        print(f"   ℹ️  Auto-generated model_name for '{model_type}': {cfg['model_name']}")
    
    # ✅ Validate that model_name has Unity Catalog format
    if not cfg["model_name"].count(".") >= 2:
        # Assume it's shorthand, convert to full path
        cfg["model_name"] = get_full_model_name(model_type)
        print(f"   ℹ️  Converted to full path: {cfg['model_name']}")
    
    # ✅ Validate required fields
    required_fields = ["version", "traffic", "active"]
    for field in required_fields:
        if field not in cfg:
            raise ValueError(f"❌ Missing '{field}' for model '{model_type}'")

print("\n✅ Configuration validated and completed")

# --------------------------------------------------
# 5️⃣ INIT CLIENTS
# --------------------------------------------------
w = WorkspaceClient()
mlflow.set_tracking_uri("databricks")
mlflow.set_registry_uri("databricks-uc")
client = MlflowClient()

# --------------------------------------------------
# 6️⃣ VERIFY MODELS EXIST IN REGISTRY
# --------------------------------------------------
print("\n🔍 Verifying models in Unity Catalog...")

for model_type, cfg in MODEL_SERVING_CONFIG.items():
    
    if not cfg.get("active", False):
        print(f"   ⏭️  Skipping inactive: {model_type}")
        continue
    
    model_name = cfg["model_name"]
    version = str(cfg["version"])
    
    try:
        # Check if model exists
        model_versions = client.search_model_versions(f"name='{model_name}'")
        
        # Check if specific version exists
        version_exists = any(mv.version == version for mv in model_versions)
        
        if not version_exists:
            available_versions = [mv.version for mv in model_versions]
            raise ValueError(
                f"❌ Version {version} not found for {model_name}\n"
                f"   Available versions: {available_versions}"
            )
        
        print(f"   ✅ Found: {model_name} v{version}")
        
    except Exception as e:
        raise ValueError(
            f"❌ Model not found in registry: {model_name}\n"
            f"   Error: {e}\n"
            f"   Make sure the model is registered first!"
        )

# --------------------------------------------------
# 7️⃣ BUILD SERVED ENTITIES (WITHOUT traffic_percentage)
# --------------------------------------------------
served_entities = []
traffic_config_routes = []

print("\n📊 Building traffic distribution...")

for model_type, cfg in MODEL_SERVING_CONFIG.items():

    if not cfg.get("active", False):
        continue

    # ✅ Generate unique name for this served entity
    served_entity_name = f"{model_type}-v{cfg['version']}"
    
    # ✅ Create ServedEntityInput WITHOUT traffic_percentage
    served_entity = ServedEntityInput(
        name=served_entity_name,  # ✅ CRITICAL: Must have unique name
        entity_name=cfg["model_name"],
        entity_version=str(cfg["version"]),
        workload_size=pipeline_cfg["serving"]["workload_size"],
        scale_to_zero_enabled=pipeline_cfg["serving"]["scale_to_zero_enabled"]
    )
    served_entities.append(served_entity)
    
    # ✅ Create Route for TrafficConfig
    route = Route(
        served_model_name=served_entity_name,
        traffic_percentage=int(cfg["traffic"])
    )
    traffic_config_routes.append(route)
    
    print(f"   • {cfg['model_name']} v{cfg['version']} → {cfg['traffic']}%")

if not served_entities:
    raise ValueError("❌ No active models found in MODEL_SERVING_CONFIG")

# ✅ Validate traffic adds up to 100%
total_traffic = sum(int(cfg["traffic"]) for cfg in MODEL_SERVING_CONFIG.values() if cfg.get("active", False))
if total_traffic != 100:
    raise ValueError(
        f"❌ Traffic percentages must add up to 100%\n"
        f"   Current total: {total_traffic}%"
    )

# ✅ Create TrafficConfig
traffic_config = TrafficConfig(routes=traffic_config_routes)

# --------------------------------------------------
# 8️⃣ ENDPOINT NAME (SINGLE PROD ENDPOINT)
# --------------------------------------------------
BASE_NAME = pipeline_cfg["models"]["base_name"]
ENDPOINT_NAME = f"{BASE_NAME}-prod"

print(f"\n🚀 Serving Endpoint: {ENDPOINT_NAME}")

# --------------------------------------------------
# 8️⃣.1️⃣ ENVIRONMENT VARIABLES (Secrets → Endpoint ENV)
# --------------------------------------------------
# These will appear in Serving Endpoint -> Edit -> Environment Variables
# Using dummy secrets from dev-scope
endpoint_env_vars = {
    "DB_USER": "{{secrets/dev-scope/db_user}}",
    "DB_PASSWORD": "{{secrets/dev-scope/db_password}}",
    "DB_HOST": "{{secrets/dev-scope/db_host}}"
}

print("\n🔐 Configuring Endpoint Environment Variables:")
for k, v in endpoint_env_vars.items():
    print(f"   • {k} = {v}")

# --------------------------------------------------
# ✅ 8️⃣.2️⃣ REST API PATCH FOR ENV VARS (ADDED)
# --------------------------------------------------
def patch_endpoint_env_vars(endpoint_name: str, env_vars: dict):
    """
    Patch endpoint config using REST API to set environment variables.
    This works even when SDK doesn't support environment_vars parameter.
    """
    import requests

    ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
    host = ctx.apiUrl().get()
    token = ctx.apiToken().get()

    url = f"{host}/api/2.0/serving-endpoints/{endpoint_name}/config"
    headers = {"Authorization": f"Bearer {token}"}

    payload = {"environment_vars": env_vars}

    r = requests.patch(url, headers=headers, json=payload)
    if 200 <= r.status_code < 300:
        print("✅ Environment variables updated via REST API")
    else:
        raise Exception(f"❌ REST API ENV update failed: {r.status_code} - {r.text}")

# --------------------------------------------------
# 9️⃣ CREATE OR UPDATE ENDPOINT
# --------------------------------------------------
def endpoint_exists(name):
    try:
        w.serving_endpoints.get(name=name)
        return True
    except:
        return False

try:
    if endpoint_exists(ENDPOINT_NAME):
        print("🔄 Updating existing endpoint...")
        w.serving_endpoints.update_config(
            name=ENDPOINT_NAME,
            config=EndpointCoreConfigInput(
                served_entities=served_entities,
                traffic_config=traffic_config  # ✅ ADDED
            )
        )
    else:
        print("➕ Creating new endpoint...")
        w.serving_endpoints.create(
            name=ENDPOINT_NAME,
            config=EndpointCoreConfigInput(
                served_entities=served_entities,
                traffic_config=traffic_config  # ✅ ADDED
            )
        )

    # ✅ PATCH ENV VARS AFTER CREATE/UPDATE
    patch_endpoint_env_vars(ENDPOINT_NAME, endpoint_env_vars)

except Exception as e:
    print("❌ Failed to deploy serving endpoint")
    traceback.print_exc()
    raise e

# --------------------------------------------------
# 🔟 WAIT UNTIL READY
# --------------------------------------------------
print("\n⏳ Waiting for endpoint to be READY...")
start = time.time()
timeout = pipeline_cfg["serving"]["deployment_timeout"]

while time.time() - start < timeout:
    ep = w.serving_endpoints.get(name=ENDPOINT_NAME)
    state = str(ep.state)

    if "READY" in state and "NOT_UPDATING" in state:
        print("✅ Endpoint is READY")
        break

    time.sleep(pipeline_cfg["serving"]["status_check_interval"])
else:
    raise TimeoutError("❌ Endpoint did not become READY in time")

# --------------------------------------------------
# 1️⃣1️⃣ SUCCESS SUMMARY
# --------------------------------------------------
print("\n" + "="*80)
print("🎉 SERVING DEPLOYMENT SUCCESSFUL")
print("="*80)
print(f"📍 Endpoint Name: {ENDPOINT_NAME}")
print(f"\n📊 Traffic Distribution:")
for route in traffic_config_routes:
    print(f"   • {route.served_model_name} → {route.traffic_percentage}%")
print("="*80)
